In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install music21

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import tensorflow
import numpy
import os
import zipfile
from music21 import instrument, note, stream, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM,ConvLSTM1D,Flatten,GRU
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import Activation
from music21 import midi
import glob
def noop(input):
  pass
midi.translate.environLocal.warn = noop

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

folders1=os.listdir('/content/data')

In [ ]:
from music21 import converter, instrument, note, chord
notes = []
for file in glob.glob('/content/data/Classical-Piano-Composer-master/midi_songs/*.mid'):
    print(file)
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
            #storing pitch of instrument
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

/content/data/Classical-Piano-Composer-master/midi_songs/balamb.mid
/content/data/Classical-Piano-Composer-master/midi_songs/In_Zanarkand.mid
/content/data/Classical-Piano-Composer-master/midi_songs/ff6shap.mid
/content/data/Classical-Piano-Composer-master/midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
/content/data/Classical-Piano-Composer-master/midi_songs/sera_.mid
/content/data/Classical-Piano-Composer-master/midi_songs/ultimafro.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Ff4-BattleLust.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Zelda_Overworld.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Rydia_pc.mid
/content/data/Classical-Piano-Composer-master/midi_songs/OTD5YA.mid
/content/data/Classical-Piano-Composer-master/midi_songs/braska.mid
/content/data/Classical-Piano-Composer-master/midi_songs/dayafter.mid
/content/data/Classical-Piano-Composer-master/midi_songs/ff4-town.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Kingdo

/usr/local/lib/python3.9/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


/content/data/Classical-Piano-Composer-master/midi_songs/Cids.mid
/content/data/Classical-Piano-Composer-master/midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Fierce_Battle_(Piano).mid
/content/data/Classical-Piano-Composer-master/midi_songs/great_war.mid
/content/data/Classical-Piano-Composer-master/midi_songs/ff4pclov.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Fiend_Battle_(Piano).mid
/content/data/Classical-Piano-Composer-master/midi_songs/FF4.mid
/content/data/Classical-Piano-Composer-master/midi_songs/HighwindTakestotheSkies.mid
/content/data/Classical-Piano-Composer-master/midi_songs/FFIXQuMarshP.mid
/content/data/Classical-Piano-Composer-master/midi_songs/AT.mid
/content/data/Classical-Piano-Composer-master/midi_songs/FF8_Shuffle_or_boogie_pc.mid
/content/data/Classical-Piano-Composer-master/midi_songs/FFIX_Piano.mid
/content/data/Classical-Piano-Composer-master/midi_songs/EyesOnMePiano.m

/usr/local/lib/python3.9/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


/content/data/Classical-Piano-Composer-master/midi_songs/8.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Fyw_piano.mid
/content/data/Classical-Piano-Composer-master/midi_songs/VincentPiano.mid
/content/data/Classical-Piano-Composer-master/midi_songs/lurk_in_dark.mid
/content/data/Classical-Piano-Composer-master/midi_songs/roseofmay-piano.mid
/content/data/Classical-Piano-Composer-master/midi_songs/mining.mid
/content/data/Classical-Piano-Composer-master/midi_songs/ff11_awakening_piano.mid
/content/data/Classical-Piano-Composer-master/midi_songs/sobf.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Rachel_Piano_tempofix.mid
/content/data/Classical-Piano-Composer-master/midi_songs/ff4_piano_collections-main_theme.mid
/content/data/Classical-Piano-Composer-master/midi_songs/Eternal_Harvest.mid
/content/data/Classical-Piano-Composer-master/midi_songs/costadsol.mid
/content/data/Classical-Piano-Composer-master/midi_songs/caitsith.mid
/content/data/Classical-Piano-

In [ ]:
from keras.utils import np_utils

In [ ]:
#Creating Sequence of 100 for each sample and generating sequence of data
sequence_length = 100
n_vocab = len(set(notes))# number of samples are set as vocab
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
#creating input and output sequence 
network_input = []
network_output = []
#addding the input sequence and output sequence from notes with look back of 100
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
#one hot encoding the target features
network_output = np_utils.to_categorical(network_output)

***LSTM model***

In [ ]:
#LSTM model
#Creating LSTM model with 1 hidden layer and 3 LSTM layers
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True)) #LSTM layer
model.add(Dropout(0.3))#dropout layer to control overfitting
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))#dropout layer to control overfitting
model.add(Dense(n_vocab))
model.add(Activation('softmax'))#output layer
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(network_input, network_output, epochs=100, batch_size=64)

Epoch 1/100
717/717 [==============================] - 17s 16ms/step - loss: 4.7356
Epoch 2/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6940
Epoch 3/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6873
Epoch 4/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6847
Epoch 5/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6823
Epoch 6/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6818
Epoch 7/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6809
Epoch 8/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6798
Epoch 9/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6803
Epoch 10/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6787
Epoch 11/100
717/717 [==============================] - 11s 16ms/step - loss: 4.6789
Epoch 12/100
717/717 [==============================] - 11s 16ms/step - lo

***GRU Model***

In [ ]:
#GRU 
model = Sequential()
model.add(GRU(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))##dropout layer to control overfitting
model.add(GRU(512, return_sequences=True)) #GRU layer
model.add(Dropout(0.3))#dropout layer to control overfitting
model.add(GRU(256))
model.add(Dense(256))
model.add(Dropout(0.3))#dropout layer to control overfitting
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Epoch 1/100
717/717 [==============================] - 16s 15ms/step - loss: 4.6053
Epoch 2/100
717/717 [==============================] - 11s 15ms/step - loss: 4.4654
Epoch 3/100
717/717 [==============================] - 11s 15ms/step - loss: 4.3084
Epoch 4/100
717/717 [==============================] - 11s 15ms/step - loss: 4.0454
Epoch 5/100
717/717 [==============================] - 11s 15ms/step - loss: 3.6519
Epoch 6/100
717/717 [==============================] - 11s 15ms/step - loss: 3.2741
Epoch 7/100
717/717 [==============================] - 11s 15ms/step - loss: 2.9863
Epoch 8/100
717/717 [==============================] - 11s 15ms/step - loss: 2.7579
Epoch 9/100
717/717 [==============================] - 11s 15ms/step - loss: 2.5694
Epoch 10/100
717/717 [==============================] - 11s 15ms/step - loss: 2.3954
Epoch 11/100
717/717 [==============================] - 11s 15ms/step - loss: 2.2526
Epoch 12/100
717/717 [==============================] - 11s 15ms/step - lo

***LSTM+GRU Model***

In [ ]:
#LSTM+GRU model
#Creating Combination of LSTM+GRU model for training 
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3)) #dropout layer to control overfitting
model.add(GRU(512, return_sequences=True)) #GRU layer
model.add(Dropout(0.3))#dropout layer to control overfitting
model.add(GRU(256))#GRU layer
model.add(Dense(256))#hidden layer
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))#output layer
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 100, 512)          1052672   
                                                                 
 dropout_9 (Dropout)         (None, 100, 512)          0         
                                                                 
 gru_3 (GRU)                 (None, 100, 512)          1575936   
                                                                 
 dropout_10 (Dropout)        (None, 100, 512)          0         
                                                                 
 gru_4 (GRU)                 (None, 256)               591360    
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dropout_11 (Dropout)        (None, 256)              

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
filepath = "/content/logs/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
hist=model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)# training LSTM+GRU model and saving weights

Epoch 1/100
717/717 [==============================] - 16s 16ms/step - loss: 4.6078
Epoch 2/100
717/717 [==============================] - 11s 16ms/step - loss: 4.4990
Epoch 3/100
717/717 [==============================] - 11s 15ms/step - loss: 4.3992
Epoch 4/100
717/717 [==============================] - 11s 16ms/step - loss: 4.2461
Epoch 5/100
717/717 [==============================] - 11s 16ms/step - loss: 4.0079
Epoch 6/100
717/717 [==============================] - 11s 16ms/step - loss: 3.6790
Epoch 7/100
717/717 [==============================] - 11s 16ms/step - loss: 3.3570
Epoch 8/100
717/717 [==============================] - 11s 16ms/step - loss: 3.0890
Epoch 9/100
717/717 [==============================] - 11s 16ms/step - loss: 2.8743
Epoch 10/100
717/717 [==============================] - 11s 16ms/step - loss: 2.6935
Epoch 11/100
717/717 [==============================] - 11s 16ms/step - loss: 2.5212
Epoch 12/100
717/717 [==============================] - 11s 16ms/step - lo

***Generating Part***

In [ ]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
#loading notes , the previous information
def generate():
    with open('/content/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
#normalizing the inputs which are randomly generated from distribution
    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)#generating notes based on input submitted (sequence lenght)
    create_midi(prediction_output)

#Preparation of 100 lengh sequence for forecasting with Newly generated data with respect to old data
def prepare_sequences(notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100#squence lengh define as per training data
    network_input = []
    output = []
    #creating loop structure to prpeare input and ouput fron extracte features 
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)#number of pattersn to be defiend
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)
#Creating Network and loading Network
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    model.load_weights('weights_lstm_gru.hdf5')#loading weights of Networks

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    start = numpy.random.randint(0, len(network_input)-1) #Different Music generation after each time you ran

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []
    for note_index in range(100): #This will control the lengh of generated music  change this number if you want to generate higher duration 
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1)) 
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)#making prediction on the data

        index = numpy.argmax(prediction)#converting the one hot encoded data to single array
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            #feature matchig of generated music and creatig a reference
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid') #saving the generated file as mid file

if __name__ == '__main__':
    generate()

In [2]:
import music21

# Load the ground truth MIDI file
gt_midi = music21.converter.parse('midi_songs/balamb.mid')

# Load the generated MIDI file
gen_midi = music21.converter.parse('test_output.mid')

# Define the evaluation metrics
evaluator = music21.evaluation.Evaluator()

# Calculate the evaluation metrics between the ground truth and generated files
results = evaluator.evaluate([gt_midi], [gen_midi])

# Print out the evaluation metrics
for metric, values in results.items():
    print(f"{metric}: {values}")


AttributeError: module 'music21' has no attribute 'Evaluator'

In [3]:
import pretty_midi
import numpy as np

# Load the ground truth and generated MIDI files
gt_midi = pretty_midi.PrettyMIDI('midi_songs/DOS.mid')
gen_midi = pretty_midi.PrettyMIDI('test_output.mid')

# Calculate the note-level metrics for the generated MIDI file
total_notes = 0
total_correct_notes = 0
for gt_instr, gen_instr in zip(gt_midi.instruments, gen_midi.instruments):
    for gt_note in gt_instr.notes:
        gt_note_start = gt_note.start
        gt_note_end = gt_note.end
        for gen_note in gen_instr.notes:
            if (gen_note.start <= gt_note_start <= gen_note.end) or \
               (gen_note.start <= gt_note_end <= gen_note.end):
                total_notes += 1
                if gt_note.pitch == gen_note.pitch:
                    total_correct_notes += 1
                break

precision = total_correct_notes / total_notes
recall = total_correct_notes / len(gt_midi.instruments[0].notes)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1_score:.3f}")


Precision: 0.022
Recall: 0.005
F1 Score: 0.008
